In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
###############################################
# 1. LOAD DATA
###############################################
df = pd.read_excel("bpt.xlsx")

# Clean column names
df.columns = df.columns.str.strip()

# Convert Time column to actual pandas datetime
df["Time"] = pd.to_datetime(df["Time"], errors="coerce")

# Replace missing values
df = df.fillna(0)

# profiles = synthetic_cycles
# capacity_drop = synthetic_cap_drop

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# 2. EXTRACT CYCLES AND CURRENT PROFILES
###############################################
profiles = []
capacity_drop = []


# Ensure cycles present
cycles = df["Cycles"].unique()
cycles = sorted([c for c in cycles if c >= 0])

for c in cycles:
    cycle_data = df[df["Cycles"] == c]
    if cycle_data.shape[0] < 5:
        continue  # skip small cycles

    # Current profile I(t)
    I = cycle_data["Current(A)"].values.astype(np.float32)

    profiles.append(I)

    # Compute ΔCapacity between cycles
    if c == 0:
        delta_c = 0
    else:
        cap_prev = df[df["Cycles"] == c-1]["Capacity(Ah)"].mean()
        cap_curr = cycle_data["Capacity(Ah)"].mean()
        delta_c = float(max(cap_prev - cap_curr, 0))

    capacity_drop.append(delta_c)


In [4]:
# 3. CUSTOM DATASET
###############################################
class BatteryStressDataset(Dataset):
    def __init__(self, profiles, capacity_drop):
        self.profiles = profiles
        self.capacity_drop = capacity_drop

    def __len__(self):
        return len(self.profiles)

    def __getitem__(self, idx):
        I = torch.tensor(self.profiles[idx], dtype=torch.float32)
        I_dot = torch.gradient(I)[0]
        profile = torch.stack([I, I_dot], dim=1)
        delta_c = torch.tensor(self.capacity_drop[idx], dtype=torch.float32)
        return profile, delta_c.unsqueeze(0)

dataset = BatteryStressDataset(profiles, capacity_drop)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [5]:
# 4. DeepONet MODEL
###############################################
class DeepONet(nn.Module):
    def __init__(self, input_dim=2, hidden=128):
        super(DeepONet, self).__init__()

        self.branch = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
        )

        self.trunk = nn.Sequential(
            nn.Linear(1, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
        )

        self.output_layer = nn.Linear(hidden, 1)

    def forward(self, profile):
        T = profile.shape[0]
        times = torch.linspace(0, 1, T).unsqueeze(-1)

        b = self.branch(profile)
        t = self.trunk(times)

        combined = b * t
        combined = torch.sum(combined, dim=0)

        return self.output_layer(combined)


In [6]:
# 5. PHYSICS-INFORMED LOSS
###############################################
def physics_informed_loss(pred, profile, target):
    I = profile[:, 0]
    I_dot = profile[:, 1]

    data_loss = nn.MSELoss()(pred, target)
    monotonicity = torch.mean(torch.relu(-pred))
    derivative_penalty = torch.mean(I_dot**2)
    current_penalty = torch.mean(torch.abs(I))

    return data_loss + 0.1*monotonicity + 0.01*current_penalty + 0.001*derivative_penalty


In [7]:
# 6. TRAINING LOOP
###############################################
model = DeepONet()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(300):
    total_loss = 0
    for profile, dc in dataloader:
        optimizer.zero_grad()
        pred = model(profile.squeeze(0))
        loss = physics_informed_loss(pred, profile.squeeze(0), dc)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss = {total_loss:.4f}")


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Loss = 46.5216
Epoch 20, Loss = 1.5854
Epoch 40, Loss = 0.2161
Epoch 60, Loss = 0.1298
Epoch 80, Loss = 0.1070
Epoch 100, Loss = 0.1064
Epoch 120, Loss = 0.1056
Epoch 140, Loss = 0.1057
Epoch 160, Loss = 0.1057
Epoch 180, Loss = 0.1057
Epoch 200, Loss = 0.1056
Epoch 220, Loss = 0.1060
Epoch 240, Loss = 0.1056
Epoch 260, Loss = 0.1056
Epoch 280, Loss = 0.1058


In [8]:
# 7. PREDICT STRESS FOR ALL CYCLES
###############################################
stress_values = []
for profile, _ in dataset:
    stress = model(profile).detach().item()
    stress_values.append(stress)

print("Predicted Stress per Cycle:")
print(stress_values)


Predicted Stress per Cycle:
[0.016455600038170815]


In [9]:
# 8. SAVE MODEL AS H5 FILE
###############################################
import h5py

def save_pytorch_model_to_h5(model, filename="deepONet_battery_stress.h5"):
    with h5py.File(filename, "w") as f:
        for name, param in model.state_dict().items():
            f.create_dataset(name, data=param.cpu().numpy())
    print(f"Model successfully saved to {filename}")

# Save model
save_pytorch_model_to_h5(model)

Model successfully saved to deepONet_battery_stress.h5
